## Demo 4 - Consequences of the Stability Theorem

In [ ]:
import time
import numpy as np

#topological data analysis
from ripser import ripser
from persim import plot_diagrams

#plotting and visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib.patches import Circle

from scipy.spatial import distance

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
 
def maxmin(dist_matrix, n):
    '''
    Given a distance matrix retunrs a maxmin subsampling and the covering radious 
    corresponding to the subsampled set.
    
    :param dist_matrix: Distance matrix
    :param n: Size of subsample set.
    :returns L: List of indices corresponding to the subsample set.
    :return cr: Covering radious for the subsample set.
    '''
    L = [np.random.randint(0,len(dist_matrix))]
    
    dist_to_L = dist_matrix[ L[-1] ,:]
    
    for i in range(n-1):
        ind = np.argmax( dist_to_L )
        L.append(ind)
        
        dist_to_L = np.minimum(dist_to_L, dist_matrix[L[-1], :])
        
    cr = np.max(dist_to_L)

    return L, cr

def uniform_sampling(dist_matrix, n):
    '''
    Given a distance matrix retunrs an subsamplig that preserves the distribution of the original data set 
    and the covering radious corresponding to the subsampled set.
    
    :param dist_matrix: Distance matrix
    :param n: Size of subsample set.
    :returns L: List of indices corresponding to the subsample set.
    :return cr: Covering radious for the subsample set.
    '''
    num_points = dist_matrix.shape[0]
    
    L = np.random.choice(num_points, n)
    
    dist_to_L = np.min(dist_matrix[L,:], axis=0)
    
    return L, np.max(dist_to_L)

### Persistence is stable with respect to Hausdorff Noise

In [ ]:
np.random.seed(2)

# Generate angles uniformly between 0 and 2pi
n_data = 400
theta = np.random.uniform(0, 2*np.pi, n_data)

# Generate radii with Gaussian noise centered at 1 and variance sigma
sigma = 0.1
r = np.random.normal(1, sigma , n_data)
X = np.array([ np.cos(theta) , np.sin(theta)]) 
Y = r*X
X = X.T
Y = Y.T

plt.figure(figsize = (5,5)) 
plt.scatter(X[:,0], X[:,1], s = 2 , label = 'data');
plt.scatter(Y[:,0], Y[:,1], s = 2, c='black' , label = 'noisy sample');
plt.legend(loc = 'lower right')
plt.axis('square');



In [ ]:
# Hausdorff distance

d_XY =  np.max(np.min(distance.cdist(X,Y), axis = 0))
d_YX =  np.max(np.min(distance.cdist(X,Y), axis = 1))

print( 'd_H(X,Y) = ',np.max( [d_XY, d_YX])  )

In [ ]:
q = 2
 
PH_X = ripser(X, coeff=q )
PH_Y = ripser(Y, coeff=q )

plt.figure(figsize = (8,3)) 
plt.subplot(1,2,1)
plot_diagrams(PH_X['dgms'])
x_left, x_right = plt.xlim()
y_left, y_right = plt.ylim()
plt.title('$PH_*(\mathcal{R}(X); \mathbb{Z}_{' + str(q) +'})$');

plt.subplot(1,2,2)
plot_diagrams(PH_Y['dgms'],  xy_range  = [x_left, x_right, y_left, y_right] )
plt.title('$PH_*(\mathcal{R}(Y); \mathbb{Z}_{' + str(q) +'})$');

plt.show()
 

## Sampling is a good idea

Generate data around the unit circle

In [ ]:
np.random.seed(2)

# Generate angles uniformly between 0 and 2pi
n_data = 800
theta = np.random.uniform(0, 2*np.pi, n_data)

# Generate radii with Gaussian noise centered at 1 and variance sigma
sigma = 0.1
r = np.random.normal(1, sigma , n_data)

X = r*np.array([ np.cos(theta) , np.sin(theta)]) 
X = X.T

# Plot data set X
plt.figure(figsize = (4,4))
plt.scatter(X[:,0], X[:,1], s = 2 );
plt.axis('square');

Persistent homology computation

In [ ]:
q = 2

start_time = time.time()
PH_X = ripser(X, coeff=q )
X_time = time.time() - start_time
print("--- %s seconds ---" % (X_time))

In [ ]:
# Plot
plt.figure(figsize=(3,3))
plot_diagrams(PH_X['dgms'])
plt.title('$PH_*(\mathcal{R}(X); \mathbb{Z}_{' + str(q) +'})$');

Uniform sample

In [ ]:
# Compute the distance matrix for the data set X using auclidean distance.
dm_X = distance.cdist(X,X)

# Subsample the data using uniform sampling
np.random.seed(2) # Comment out to get different samples
n_landmarks = 40
ind_Y, cover_r = uniform_sampling(dm_X, n_landmarks)
Y = X[ind_Y]

# scatter plot the landmark subset
fig, ax = plt.subplots(figsize = (4,4))
plt.scatter(X.T[0], X.T[1], s = 2)
plt.scatter(Y.T[0], Y.T[1], s = 5, c='red')

# Plot balls or radious defined by the covering radious of the landmark set
for i in range(n_landmarks):
    cir = Circle((Y[i,0], Y[i,1]), cover_r, color=(0,0,0,0.05))
    ax.add_patch(cir)
ax.set_ylim(-1-2*cover_r,1+2*cover_r);
ax.set_xlim(-1-2*cover_r,1+2*cover_r);

r_Y = cover_r
print('d_H(X,Y) =',  r_Y)

In [ ]:
q = 2

start_time = time.time()
PH_Y = ripser(Y, coeff=q )
Y_time = time.time() - start_time
print("--- X_time = %s seconds ---" % (X_time))
print("--- Y_time = %s seconds ---" % (Y_time))


In [ ]:
plt.figure(figsize = (8,3)) 
plt.subplot(1,2,2)
plot_diagrams(PH_Y['dgms'])
plt.title('$PH_*(\mathcal{R}(Y); \mathbb{Z}_{' + str(q) +'})$');
x_left, x_right = plt.xlim()
y_left, y_right = plt.ylim()

plt.subplot(1,2,1)
plot_diagrams(PH_X['dgms'], xy_range  = [x_left, x_right, y_left, y_right] )
plt.title('$PH_*(\mathcal{R}(X); \mathbb{Z}_{' + str(q) +'})$');
plt.show()

In [ ]:
# Subsample the data using MaxMin sampling
np.random.seed(2) # Comment out to get different samples
ind_M, cover_r = maxmin(dm_X, n_landmarks)
M = X[ind_M]

# scatter plot the landmark subset
fig, ax = plt.subplots(figsize = (4,4))
plt.scatter(X.T[0], X.T[1], s = 2)
plt.scatter(M.T[0], M.T[1], s = 5, c='red')

# Plot balls or radious defined by the covering radious of the landmark set
for i in range(n_landmarks):
    cir = Circle((M[i,0], M[i,1]), cover_r, color=(0,0,0,0.05))
    ax.add_patch(cir)
ax.set_ylim(-1-2*cover_r,1+2*cover_r);
ax.set_xlim(-1-2*cover_r,1+2*cover_r);

print('d_H(X,Y) =', r_Y)
print('d_H(X,M) =',  cover_r)


In [ ]:
q = 2
start_time = time.time()
PH_M= ripser(M, coeff=q )
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
plt.figure(figsize = (12,3)) 
plt.subplot(1,3,2)
plot_diagrams(PH_Y['dgms'])
plt.title('$PH_*(\mathcal{R}(Y); \mathbb{Z}_{' + str(q) +'})$');
x_left, x_right = plt.xlim()
y_left, y_right = plt.ylim()

plt.subplot(1,3,1)
plot_diagrams(PH_X['dgms'], xy_range  = [x_left, x_right, y_left, y_right] )
plt.title('$PH_*(\mathcal{R}(X); \mathbb{Z}_{' + str(q) +'})$');

plt.subplot(1,3,3)
plot_diagrams(PH_M['dgms'], xy_range  = [x_left, x_right, y_left, y_right] )
plt.title('$PH_*(\mathcal{R}(M); \mathbb{Z}_{' + str(q) +'})$');
plt.show()

---

## Experiment: 

The goal of this experiment is to explore the effect that data size has on the computation time of persistent homology.

1. Select subsamples of different sizes, say from 5 to 500 in increments of 5.
2. Compute persistent homology for each subsample set generated, and compute the time ripser takes to compute persistent homology in each case.  
3. Plot sample size vs time and explain the effect the sample size has on the computation time.

In [ ]:
q = 2

ripser_time =[]

for i in range(5,505,5):
    ind_M, cover_r = maxmin(dm_X, i)
    M = X[ind_M]
    
    t0 = time.time()
    per_homology = ripser(M, coeff=q)
    t1 = time.time()
    
    ripser_time.append(t1-t0)


plt.plot(ripser_time)

plt.xticks(np.arange(0,101,20), np.arange(5,506,100), color='k', size=12)
plt.xlabel('Subsample size')

plt.ylabel('Time (s)')

plt.show()

### Hausdorff vs Non-Hausdorff Noise

In [ ]:
n_noise = 20
np.random.seed(2)
theta = np.random.uniform(0, 2*np.pi, n_noise)

# Generate radii with Gaussian noise centered at 1 and variance sigma
r = np.random.uniform(-1, 1 , n_noise)

D = r*np.array([ np.cos(theta) , np.sin(theta)]) 
D = D.T

Y = np.vstack((X,D))

# Plot data set X

plt.figure(figsize = (8,3)) 
plt.subplot(1,2,1)
plt.scatter(X[:,0], X[:,1], s = 2 );
plt.axis('square');
plt.subplot(1,2,2)
plt.scatter(Y[:,0], Y[:,1], s = 2 );
plt.axis('square');


In [ ]:
# Hausdorff distance

d_XY =  np.max(np.min(distance.cdist(X,Y), axis = 0))
d_YX =  np.max(np.min(distance.cdist(X,Y), axis = 1))

print( 'd_H(X,Y) = ',np.max( [d_XY, d_YX])  )

In [ ]:
#Persistence computation
PH_Y = ripser(Y, coeff=q )

plt.figure(figsize = (8,3)) 
plt.subplot(1,2,1)
plot_diagrams(PH_X['dgms'])
x_left, x_right = plt.xlim()
y_left, y_right = plt.ylim()
plt.title('$PH_*(\mathcal{R}(X); \mathbb{Z}_{' + str(q) +'})$');

plt.subplot(1,2,2)
plot_diagrams(PH_Y['dgms'],  xy_range  = [x_left, x_right, y_left, y_right] )
plt.title('$PH_*(\mathcal{R}(Y); \mathbb{Z}_{' + str(q) +'})$');

plt.show()
 